# Crashes on a Street

In [1]:
import json
import pytz
import folium
from folium.features import DivIcon
from folium.plugins import HeatMap

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib._color_data import TABLEAU_COLORS

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
most_recent_crash = cda.most_recent_crash_timestamp()

## Find ID of street

In [3]:
query = """
select
sr.gid
, sr.lrsroute
, st_AsGeoJSON(sr.geom) as street_line
, sc.fullname
, count(distinct sc.masterid) as count_of_masterids
, row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
    as fullname_priority

from street_routes sr
inner join street_centerline sc using (lrsroute)

group by 1,2,3,4
"""

street_ids = pd.read_sql(query, cda.conn)

In [4]:
streets_to_map = street_ids[street_ids.lrsroute.str.contains('14TH')].reset_index()
streets_to_map

,index,gid,lrsroute,street_line,fullname,count_of_masterids,fullname_priority
0,101,100,114TH AV,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 114TH AVE,4,1
1,102,101,114THS AV-I,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",NORTH PERIMETER RD,4,1
2,126,125,14TH AV,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 14TH AVE,95,1
3,4161,4062,14TH ST1,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",14TH ST,3,1
4,4162,4063,14TH ST2,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",14TH ST,13,1
5,4377,4277,W14TH AV1,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 14TH AVE,21,1
6,4378,4278,W14TH AV2,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 14TH AVE,4,1
7,4379,4279,W14TH AV3,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 14TH AVE,1,1
8,4380,4280,W14TH AV4,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 14TH AVE,2,1
9,4381,4281,W14TH AV5,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 14TH AVE,2,1


In [5]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

street_color = TABLEAU_COLORS['tab:orange']

for idx, row in streets_to_map.iterrows():
    folium.GeoJson(row.street_line, style_function=lambda x: {'color': street_color}).add_to(this_map)

    street_line_dict = json.loads(row.street_line)
    middle_point = int(np.round(len(street_line_dict['coordinates'][0]) / 2))
    
    folium.map.Marker(
        [street_line_dict['coordinates'][0][middle_point][1], street_line_dict['coordinates'][0][middle_point][0]]
        , icon=DivIcon(
            icon_size=(250,36)
            , icon_anchor=(0,0)
            , html=f'<div style="font-size: 20pt; color:{street_color}">gid: {row.gid}</div>'
        )
     ).add_to(this_map)

this_map.fit_bounds(this_map.get_bounds())
this_map

## Heatmap of Crashes on One Street

In [6]:
# street_gid = 125 # E 14th
street_gid = 3900 # Speer

In [7]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))

# end_date = pytz.timezone('America/Denver').localize(datetime(2023,6,1))
end_date = pytz.timezone('America/Denver').localize(datetime.now())

In [8]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected
, c.incident_id

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where sr.gid = {street_gid}
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [9]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,209,0,3,4,2
2014,226,0,3,2,5
2015,235,0,3,6,1
2016,207,1,2,3,6
2017,222,0,3,5,6
2018,291,0,5,8,5
2019,278,0,9,2,4
2020,155,0,2,1,1
2021,145,2,2,2,1


In [10]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
days_in_data = (end_date - start_date).days

In [12]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

1.6991150442477876

In [13]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

1.5636527339164645

In [14]:
street_crashes.sort_values(by='reported_date').tail(10)

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected,incident_id
9,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-105.007438,39.752419,2023-06-23 22:48:00+00:00,2023-06-23 16:48:00.000000,N SPEER BLVD / ELITCH CIR,2023333351
8,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-105.004851,39.750138,2023-06-24 04:32:00+00:00,2023-06-23 22:32:00.000000,WEWATTA ST / N SPEER BLVD,2023333738
7,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-104.998843,39.741401,2023-06-24 05:17:00+00:00,2023-06-23 23:17:00.000000,N SPEER BLVD / STOUT ST,2023333903
6,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-105.015837,39.756302,2023-06-26 16:23:00+00:00,2023-06-26 10:23:00.000000,N ZUNI ST / N SPEER BLVD,2023338688
5,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-104.990654,39.729976,2023-06-29 17:45:00+00:00,2023-06-29 11:45:00.000000,N SPEER BLVD / N BANNOCK ST,2023344846
4,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-104.993517,39.733865,2023-06-29 19:48:00+00:00,2023-06-29 13:48:00.000000,W 11TH AVE / N SPEER BLVD,2023345129
3,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-104.989936,39.729045,2023-06-30 22:57:00.000001+00:00,2023-06-30 16:57:00.000001,W 8TH AVE / N SPEER BLVD,2023347316
2,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-104.999338,39.742312,2023-07-01 09:46:00+00:00,2023-07-01 03:46:00.000000,N SPEER BLVD / CHAMPA ST,2023348545
1,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-105.015837,39.756302,2023-07-01 22:56:00+00:00,2023-07-01 16:56:00.000000,N SPEER BLVD / N ZUNI ST,2023349349
0,3900,SPEERN BD,2.634723,False,False,0.0,0.0,-105.003227,39.748503,2023-07-03 07:56:00+00:00,2023-07-03 01:56:00.000000,AURARIA PKWY / N SPEER BLVD,2023352666


In [15]:
cda.conn.dispose()

In [16]:
len(street_crashes)

2260